In [9]:
# OPTIONAL: Load the "autoreload" extension so that code can change
%load_ext autoreload

# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
import json
import math
import os
import re
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd
import requests
import seaborn as sns
from tqdm.auto import tqdm

In [11]:
from src.data import *

In [4]:
project_dir = Path(os.path.abspath('')).parents[1]
data_dir = project_dir / "data"

In [7]:
articles = pd.read_csv(data_dir / "interim/structured.csv", index_col="article_id")

crossref = pd.read_csv(data_dir / "interim/_crossref.csv", index_col="article_id")
altmetric = pd.read_csv(data_dir / "interim/_altmetric.csv", index_col="article_id")

/home/asura/Projects/scholcommlab/military-grants/.venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (50,52,53,61) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
crossref

,indexed,edition-number,reference-count,publisher,isbn-type,license,content-domain,published-print,DOI,type,...,accepted,abstract,subtype,content-created,update-to,approved,degree,content-updated,review,short-title
article_id,,,,,,,,,,,,,,,,,,,,,
0,"{'date-parts': [[2019, 12, 4]], 'date-time': '...",1.0,89,Wiley,"[{'value': '9781118294871', 'type': 'print'}, ...",[{'URL': 'http://doi.wiley.com/10.1002/tdm_lic...,"{'domain': [], 'crossmark-restriction': False}","{'date-parts': [[2014, 3, 21]]}",10.1002/9781118294895.ch50,other,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"{'date-parts': [[2019, 11, 27]], 'date-time': ...",NaN,47,Wiley,NaN,[{'URL': 'http://doi.wiley.com/10.1002/tdm_lic...,"{'domain': [], 'crossmark-restriction': False}","{'date-parts': [[2017, 12]]}",10.1002/jts.22239,journal-article,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"{'date-parts': [[2019, 11, 26]], 'date-time': ...",NaN,27,Guilford Publications,NaN,NaN,"{'domain': [], 'crossmark-restriction': False}","{'date-parts': [[2017, 6]]}",10.1521/bumc.2017.81.2.107,journal-article,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"{'date-parts': [[2020, 1, 30]], 'date-time': '...",NaN,67,Public Library of Science (PLoS),NaN,[{'URL': 'http://creativecommons.org/licenses/...,"{'domain': ['www.plosone.org'], 'crossmark-res...",NaN,10.1371/journal.pone.0157568,journal-article,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"{'date-parts': [[2020, 2, 13]], 'date-time': '...",NaN,46,Elsevier BV,NaN,NaN,"{'domain': ['elsevier.com', 'sciencedirect.com...","{'date-parts': [[2013, 5]]}",10.1016/j.janxdis.2013.04.003,journal-article,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18703,"{'date-parts': [[2019, 12, 18]], 'date-time': ...",NaN,36,Springer Science and Business Media LLC,NaN,"[{'URL': 'http://www.springer.com/tdm', 'start...","{'domain': [], 'crossmark-restriction': False}","{'date-parts': [[1999, 2]]}",10.1038/sj.bjc.6690114,journal-article,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18704,"{'date-parts': [[2019, 11, 27]], 'date-time': ...",NaN,65,Elsevier BV,NaN,[{'URL': 'https://www.elsevier.com/tdm/userlic...,"{'domain': ['elsevier.com', 'sciencedirect.com...","{'date-parts': [[2016, 9]]}",10.1016/j.phrs.2016.07.044,journal-article,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18705,"{'date-parts': [[2019, 11, 26]], 'date-time': ...",NaN,0,American Society for Microbiology,NaN,NaN,"{'domain': [], 'crossmark-restriction': False}","{'date-parts': [[1998, 12, 1]]}",10.1128/mcb.18.12.7602,journal-article,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
p = pubs[pubs.CR_DOI_score.notna()]
p.head()

,proposal_id,unstruct_citation,type,DOI,PMCID,PMID,URL,year,container-title,title,author_str,CR_DOI_candidate,CR_DOI_score
article_id,,,,,,,,,,,,,
0,8154001,Stanley EA and Jha AP. 2009. Mind fitness and ...,PH_TBI,NaN,NaN,NaN,NaN,2009.0,Joint Force Quarterly,Mind fitness and mental armor: Enhancing perfo...,"EA, Stanley; AP, Jha",10.1002/9781118294895.ch50,43.241447
1,8214003,"Gramlich MA, Neer SM, Beidel DC, et al. 2017. ...",PH_TBI,10.1002/jts.22239,NaN,NaN,NaN,2017.0,J Trauma Stress,Posttraumatic Stress Disorder or Combat Experi...,"Gramlich, M.A.; Neer, S.M.; Beidel, D.C.",10.1002/jts.22239,122.720480
2,8214003,"Beidel DC, Stout JW, Neer SM,et al. 2017. An i...",PH_TBI,10.1521/bumc.2017.81.2.107,NaN,NaN,NaN,2017.0,Bulletin of the Menninger Clinic,An intensive outpatient treatment program for ...,"Beidel DC, Stout JW, Neer SM,et al.",10.1521/bumc.2017.81.2.107,123.194664
3,8214003,"Munyan BG, Neer SM, Beidel DC, et al. 2016. Ol...",PH_TBI,10.1371/journal.pone.0157568,4910977.0,NaN,NaN,2016.0,PLoS One,Olfactory stimuli increase presence in virtual...,"Munyan, B.G.; Neer, S.M.; Beidel, D.C.",10.1371/journal.pone.0157568,115.209380
4,8214003,"Ragsdale KA, Neer SM, Beidel DC, et al. 2013. ...",PH_TBI,NaN,NaN,NaN,NaN,2013.0,Journal of Anxiety Disorders,Posttraumatic stress disorder in OEF/OIF veter...,"Ragsdale, K.A.; Neer, S.M.; Beidel, D.C.",10.1016/j.janxdis.2013.04.003,116.886635


In [54]:
x.columns

Index(['title', 'doi', 'pmid', 'isbns', 'altmetric_jid', 'issns', 'journal',
       'cohorts', 'abstract', 'context', 'authors', 'type', 'handles',
       'altmetric_id', 'schema', 'is_oa', 'publisher_subjects',
       'cited_by_posts_count', 'cited_by_msm_count', 'cited_by_tweeters_count',
       'cited_by_accounts_count', 'last_updated', 'score', 'history', 'url',
       'added_on', 'published_on', 'subjects', 'scopus_subjects', 'readers',
       'readers_count', 'images', 'details_url', 'pmc', 'ads_id', 'uri',
       'abstract_source', 'cited_by_patents_count', 'cited_by_feeds_count',
       'cited_by_fbwalls_count', 'cited_by_videos_count',
       'cited_by_wikipedia_count', 'cited_by_gplus_count',
       'cited_by_rdts_count', 'cited_by_policies_count', 'cited_by_rh_count',
       'cited_by_peer_review_sites_count', 'editors', 'ordinal_number',
       'arxiv_id', 'cited_by_qna_count', 'cited_by_weibo_count', 'attribution',
       'hlom_id', 'hollis_id', 'authors_or_editors', 'hand

In [56]:
x[['cited_by_tweeters_count', 'cited_by_fbwalls_count', 'cited_by_wikipedia_count', 'cited_by_videos_count', 'cited_by_peer_review_sites_count']].describe()

,cited_by_tweeters_count,cited_by_fbwalls_count,cited_by_wikipedia_count,cited_by_videos_count,cited_by_peer_review_sites_count
count,2602.000000,744.000000,1389.000000,84.000000,155.0
mean,7.977709,4.174731,1.413247,1.738095,1.0
std,25.261042,56.415008,1.594026,2.380235,0.0
min,1.000000,1.000000,1.000000,1.000000,1.0
25%,1.000000,1.000000,1.000000,1.000000,1.0
50%,2.000000,1.000000,1.000000,1.000000,1.0
75%,6.000000,2.000000,1.000000,1.000000,1.0
max,489.000000,1538.000000,36.000000,19.000000,1.0


In [60]:
len(x)

8589